In [23]:
import tensorflow as tf
import matplotlib.pyplot as plt

import numpy as np
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv2DTranspose, Conv2D, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

# Define the generator for GAN
def build_generator():
    model = Sequential()
    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=100))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding="same", activation="tanh"))

    noise = Input(shape=(100,))
    img = model(noise)

    return Model(noise, img)

# Define the discriminator for GAN
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(28, 28, 1), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=(28, 28, 1))
    validity = model(img)

    return Model(img, validity)

# Define the combined model for training the generator with the discriminator
def build_gan(generator, discriminator):
    optimizer = Adam(0.0002, 0.5)
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    discriminator.trainable = False
    noise = Input(shape=(100,))
    img = generator(noise)
    validity = discriminator(img)
    combined = Model(noise, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    return combined

# Load the MNIST dataset
(X_train, _), (_, _) = mnist.load_data()

# Normalize the data
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

In [34]:


# Function to save the generated images
def save_imgs(generator, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Clip the generated images to the range [-1, 1]
    gen_imgs = np.clip(gen_imgs, -1, 1)

    # Rescale the images to 0-1 range
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Create a grid of images
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(f"images/mnist_{epoch}.png")
    plt.close()


In [36]:
# Train the GAN
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)
batch_size = 64
epochs = 10
save_interval = 1000

# Build the optimizer with the full list of trainable variables
optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

# Build the discriminator and generator loss functions
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

for epoch in range(epochs):
    # Generate a batch of noise samples
    noise = np.random.normal(0, 1, (batch_size, 100))

    # Generate a batch of fake images
    gen_imgs = generator.predict(noise)

    # Select a random batch of images from the training data
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    # Convert the NumPy array to a TensorFlow tensor
    imgs = tf.convert_to_tensor(imgs, dtype=tf.float32)

    # Create a batch of adversarial examples using FGSM
    eps = 0.1
    with tf.GradientTape() as tape:
        tape.watch(imgs)
        preds = discriminator(imgs)
    grads = tape.gradient(preds, imgs)
    adv_x = imgs + eps * tf.sign(grads)
    adv_x = tf.clip_by_value(adv_x, -1, 1)

    # Train the discriminator on the adversarial examples
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 1)))
    d_loss_adv = discriminator.train_on_batch(adv_x, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(0.5 * np.add(d_loss_real, d_loss_fake), d_loss_adv)

    # Train the generator using the combined model
    with tf.GradientTape() as tape:
        gen_imgs = generator(noise)
        preds = discriminator(gen_imgs)
        g_loss = loss_fn(tf.ones_like(preds), preds)
    grads = tape.gradient(g_loss, generator.trainable_variables)
    optimizer.apply_gradients(zip(grads, generator.trainable_variables))

    # Print the progress
    print(f"Epoch {epoch}/{epochs}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")

    # Save the generated images periodically
    if epoch % save_interval == 0:
        save_imgs(generator, epoch)


2/2 [==============================] - 0s 76ms/step
Epoch 0/10, Discriminator Loss: 1.5525078475475311, Generator Loss: 0.6638476848602295
2/2 [==============================] - 0s 78ms/step
Epoch 1/10, Discriminator Loss: 1.1283044815063477, Generator Loss: 0.6772086024284363
2/2 [==============================] - 0s 68ms/step
Epoch 2/10, Discriminator Loss: 0.9819150269031525, Generator Loss: 0.6999610662460327
2/2 [==============================] - 0s 94ms/step
Epoch 3/10, Discriminator Loss: 0.8671612665057182, Generator Loss: 0.7099773287773132
2/2 [==============================] - 0s 109ms/step
Epoch 4/10, Discriminator Loss: 0.853506475687027, Generator Loss: 0.7043718099594116
2/2 [==============================] - 0s 186ms/step
Epoch 5/10, Discriminator Loss: 0.9290437735617161, Generator Loss: 0.6945278644561768
2/2 [==============================] - 0s 63ms/step
Epoch 6/10, Discriminator Loss: 0.8831215165555477, Generator Loss: 0.6707659959793091
2/2 [=====================

In [37]:
generator.save_weights("generator.h5")

In [39]:
# Load the test data
(X_test, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_test = (X_test.astype(np.float32) - 127.5) / 127.5
X_test = np.expand_dims(X_test, axis=3)

# Generate new images for the test data using the generator
generator = build_generator()
generator.load_weights("generator.h5")
generator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))


noise = np.random.normal(0, 1, (X_test.shape[0], 100))
gen_imgs = generator.predict(noise)

# Rescale the images to 0-1 range
gen_imgs = 0.5 * gen_imgs + 0.5

# Evaluate the generator on the test data
score = generator.evaluate(noise, gen_imgs)
print("Generator score:", score)

1875/1875 [==============================] - 134s 71ms/step - loss: 3.6417
Generator score: 3.641691207885742
